### Add Imports

In [23]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Read dataset and print the shapes

In [6]:
df = pd.read_csv('train.csv')

feature_names = df.columns
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values

In [7]:
print("Feature Names: ")
print(feature_names)
print("Features Matrix (X):")
print(X[:2])
print("\nTarget Vector (Y):")
print(Y[:2])

Feature Names: 
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCo

### See features

##### See Unique values per column

In [ ]:
def create_ranges(series, num_bins=20):
    min_val, max_val = series.min(), series.max()
    range_step = (max_val - min_val) / num_bins
    ranges = [f"{min_val + range_step * i:.2f} - {min_val + range_step * (i+1):.2f}" for i in range(num_bins)]
    return ranges

def print_ranges(df):
  for column in df.columns:
      unique_values = df[column].unique()

      if unique_values.size > 20 and np.issubdtype(df[column].dtype, np.number):
            print(f"Column '{column}' has more than 20 unique numeric values. Creating ranges:")
            print(create_ranges(df[column]))
      else:
        print(f"Unique values in '{column}':")
        print(unique_values)
      print()

print_ranges(df)

##### Check for Nans

In [38]:
def replaceNans():
  df_numeric = df.select_dtypes(include=[np.number])
  df[df_numeric.columns] = df_numeric.fillna(0)

replaceNans()

### Feature Engineering

In [39]:
numerical_data = df.select_dtypes(include=['int64', 'float64'])
categorical_data = df.select_dtypes(include=['object'])

# One hot encoding for categories
categorical_data_encoded = pd.get_dummies(categorical_data)

# Apply Min-Max Scaling to numerical data
scaler = MinMaxScaler()
numerical_data_scaled = scaler.fit_transform(numerical_data)
numerical_data_scaled_df = pd.DataFrame(numerical_data_scaled, columns=numerical_data.columns)

preprocessed_df = pd.concat([numerical_data_scaled_df, categorical_data_encoded], axis=1)

print(preprocessed_df)

            Id  MSSubClass  LotFrontage   LotArea  OverallQual  OverallCond  \
0     0.000000    0.235294     0.207668  0.033420     0.666667        0.500   
1     0.000685    0.000000     0.255591  0.038795     0.555556        0.875   
2     0.001371    0.235294     0.217252  0.046507     0.666667        0.500   
3     0.002056    0.294118     0.191693  0.038561     0.666667        0.500   
4     0.002742    0.235294     0.268371  0.060576     0.777778        0.500   
...        ...         ...          ...       ...          ...          ...   
1455  0.997258    0.235294     0.198083  0.030929     0.555556        0.500   
1456  0.997944    0.000000     0.271565  0.055505     0.555556        0.625   
1457  0.998629    0.294118     0.210863  0.036187     0.666667        1.000   
1458  0.999315    0.000000     0.217252  0.039342     0.444444        0.625   
1459  1.000000    0.000000     0.239617  0.040370     0.444444        0.625   

      YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinS

In [ ]:
print_ranges(preprocessed_df)

In [40]:
preprocessed_df.drop('Id', axis=1, inplace=True)

#### Train

In [41]:
input_shape = (len(preprocessed_df.columns),)  # Replace 'number_of_features' with the actual number

# Create a Sequential model
model = Sequential([
    Dense(1, input_shape=input_shape)  # No activation function for regression
])

# Compile the model for regression
model.compile(optimizer='adam',
              loss='mean_squared_error',  # Common choice for regression
              metrics=['mae'])  # Mean Absolute Error as an additional metric

In [42]:
model.fit(preprocessed_df, Y, epochs=100, batch_size=32)


Epoch 1/100
46/46 [==============================] - 1s 2ms/step - loss: 39039025152.0000 - mae: 180920.5156
Epoch 2/100
46/46 [==============================] - 0s 3ms/step - loss: 39038230528.0000 - mae: 180918.3594
Epoch 3/100
46/46 [==============================] - 0s 2ms/step - loss: 39037431808.0000 - mae: 180916.2344
Epoch 4/100
46/46 [==============================] - 0s 2ms/step - loss: 39036633088.0000 - mae: 180914.0781
Epoch 5/100
46/46 [==============================] - 0s 2ms/step - loss: 39035834368.0000 - mae: 180911.9062
Epoch 6/100
46/46 [==============================] - 0s 2ms/step - loss: 39035043840.0000 - mae: 180909.7812
Epoch 7/100
46/46 [==============================] - 0s 2ms/step - loss: 39034245120.0000 - mae: 180907.6094
Epoch 8/100
46/46 [==============================] - 0s 2ms/step - loss: 39033450496.0000 - mae: 180905.4375
Epoch 9/100
46/46 [==============================] - 0s 2ms/step - loss: 39032647680.0000 - mae: 180903.2656
Epoch 10/100
46/46 